In [ ]:
import numpy as np
import sympy

# Importing necessary quantum computing library (QISKIT)
import qiskit
from qiskit import transpile, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import *

import torch

## Coding the data generation with a qiskit twist

In [ ]:

# https://www.tensorflow.org/quantum/tutorials/qcnn
# This function is a readaptation of the tensorflow tutorial using qiskit and pytorch instead
# WARNING: I believe the function is working, but it looks like it is not possible to convert quantum gates
# and quantum circuits to pytorch tensors.
# I don't think there is an equivalent of tfq.convert_to_tensor yet.
# For the moment the function only returns a tuple of lists for train and test excitations.
def generate_data(qubits):
    """Generate training and testing data."""
    n_rounds = 20  # Produces n_rounds * n_qubits datapoints.
    excitations = []
    labels = []
    for n in range(n_rounds):
        for bit in qubits:
            rng = np.random.uniform(-np.pi, np.pi)
            # Creating a quantum circuit with qiskit
            excitations.append(QuantumCircuit(bit.register).rx(rng, bit.register))
            #excitations.append(cirq.Circuit(cirq.rx(rng)(bit))) / cirq to check if it's correct
            labels.append(1 if (-np.pi / 2) <= rng <= (np.pi / 2) else -1)

    split_ind = int(len(excitations) * 0.7)
    train_excitations = excitations[:split_ind]
    test_excitations = excitations[split_ind:]

    train_labels = labels[:split_ind]
    test_labels = labels[split_ind:]
    
    return train_excitations, np.array(train_labels), \
        test_excitations, np.array(test_labels)


In [ ]:
train_excitations, train_labels, test_excitations, test_labels = generate_data(qr)

## Coding the cluster-state

In [ ]:
# Source:https://www.tensorflow.org/quantum/tutorials/qcnn
def cluster_state(qr):
    
    qc = QuantumCircuit(qr) # Creating a QuantumRegister
    
    # Applying a Hadamard gate to qubits
    for i, _ in enumerate(qr): 
        qc.h(i)
        
    for this_bit, next_bit in zip(qr, qr[1:] + [qr[0]]):
        c = this_bit.index
        t = next_bit.index
        qc.cz(c, t)
    return qc

In [ ]:
qc_cs = cluster_state(QuantumRegister(4))

In [ ]:
qc_cs.draw('mpl')

## QCNN layers

### one qubit unitary

In [ ]:
# Source of the function: https://www.tensorflow.org/quantum/tutorials/qcnn
# The function has been re-adapted for qiskit use
from qiskit.circuit import Parameter
def one_qubit_unitary(bit, rotation=('1', '2', '3')):
    """Make a circuit enacting a rotation of the bloch sphere about the X,
    Y and Z axis, that depends on the values in `symbols`.
    Parameters
    -----------
    bit: (QuantumRegister (qubit)) 
        qubit that to rotate
    rotation: (tuple) 
        tuple containing the three rotation angle for respectively, x, y and z
    Returns
    -------
        Rotated qubit
    """
    x, y, z = rotation
    qc = QuantumCircuit(bit)
    qc.rx(Parameter(x), 0)
    qc.ry(Parameter(y), 0)
    qc.rz(Parameter(z), 0)
    return qc

In [ ]:
one_qubit_unitary(1, ("e1", "e2", "e3")).draw()

### two qubit unitary

In [ ]:
# Function still needs some rechecking but it might be correct.
# replace symbols later, still having an issue with symbols
def two_qubit_unitary(bits): 
    """Make a Cirq circuit that creates an arbitrary two qubit unitary."""

    sub_circ1 = one_qubit_unitary(1)
    sub_circ2 = one_qubit_unitary(1, ('x2', 'y2', 'z2'))

    qr = bits
    big_qc = QuantumCircuit(qr)
    big_qc.append(sub_circ1.to_instruction(), [qr[0]])
    big_qc.append(sub_circ2.to_instruction(), [qr[1]])
    big_qc.rzz(Parameter('theta2'), 0, 1)
    big_qc.ryy(Parameter('theta3'), 0, 1)
    big_qc.rxx(Parameter('theta4'), 0, 1)
    big_qc.append(sub_circ1.to_instruction(), [qr[0]])
    big_qc.append(sub_circ2.to_instruction(), [qr[1]])
    
    return big_qc

In [ ]:
two_qubit_unitary(QuantumRegister(2)).decompose().draw('mpl')

### two qubit pool

In [ ]:
# source: https://www.tensorflow.org/quantum/tutorials/qcnn
def two_qubit_pool(source_qubit, sink_qubit): # add symbols later
    """Make a Qiskit circuit to do a parameterized 'pooling' operation, which
    attempts to reduce entanglement down from two qubits to just one."""
    
    sink_basis_selector = one_qubit_unitary(sink_qubit)
    source_basis_selector = one_qubit_unitary(source_qubit, ('x2', 'y2', 'z2'))
    
    qr = QuantumRegister(2)
    pool_circuit = QuantumCircuit(qr)
    pool_circuit.append(sink_basis_selector.to_instruction(), [qr[0]])
    pool_circuit.append(source_basis_selector.to_instruction(), [qr[1]])
    pool_circuit.cnot(control_qubit=0, target_qubit=1)
    
    # add sink_basis selector I don't know what is being done
    inv_sink_basis_selector = one_qubit_unitary(source_qubit, ('-x', '-y', '-z'))
    pool_circuit.append(inv_sink_basis_selector.to_instruction(), [qr[1]])
    
    return pool_circuit

In [ ]:
two_qubit_pool(QuantumRegister(1), QuantumRegister(1)).decompose().draw('mpl')

In [ ]:
# source: https://www.tensorflow.org/quantum/tutorials/qcnn
def quantum_conv_circuit(bits, symbols):
    """Quantum Convolution Layer following the above diagram.
    Return a Cirq circuit with the cascade of `two_qubit_unitary` applied
    to all pairs of qubits in `bits` as in the diagram above.
    """
    circuit = cirq.Circuit()
    for first, second in zip(bits[0::2], bits[1::2]):
        circuit += two_qubit_unitary([first, second], symbols)
    for first, second in zip(bits[1::2], bits[2::2] + [bits[0]]):
        circuit += two_qubit_unitary([first, second], symbols)
    return circuit

In [ ]:
def quantum_conv_circuit(bits):
    
    qc = QuantumCircuit(bits)
    #qc.append(two_qubit_unitary(QuantumRegister(2)).to_instruction(), [first, second])

    for first, second in zip(bits[0::2], bits[1::2]):
        print(first)
        test = two_qubit_unitary(first.register)
        qc.append(test.to_instruction(), [first])
        #test2 = two_qubit_unitary(second.register)
    return test

In [ ]:
quantum_conv_circuit(QuantumRegister(8)).draw('mpl')